## Projeto de extração e análise de dados de Fundos de Investimentos Brasileiros

#### Os fundos de investimento têm atraído cada vez mais pessoas que querem sair da poupança em busca de uma alternativa mais rentável ou diversificar os investimentos sem se preocupar em administrá-los em seus minímos detalhes. Porém a tarefa de escolhê-los está cada vez mais dificil. Por termos muitas opções é normal ficarmos perdidos, sem saber a melhor opção para alocarmos nossos investimentos.

#### Com isso em mente preparei um estudo que visa clarificar este mercado para que possamos tomar as melhores descisões!

#### O primeiro passo do nosso projeto consiste na extração e transformação dos dados de todos os FI brasileiros. Para isso utilizamos as seguintes fontes:


- Portal Dados Abertos CVM:
    No portal da CVM encontrei as cotas diárias de todos os fundos ativos no brasil desde 2005, com precisão diária.
    Porém o único dado que temos de cada fundo é o seu CNPJ.
    Fonte: http://dados.cvm.gov.br/dataset/fi-doc-inf_diario


- financial.io:
    Com o API da financial.io consegui mais informações sobre cada fundo, como nome, situação, classe de investimentos,
    situação e gestor.
    Fonte: https://financialdata.io/fundos


- Debit.com.br:
    - É importante também que tenhamos em mãos os benchmarks para analisarmos os fundos. Do site da Debit pude recolher 
    os dados de CDI (variação mensal) desde 1986 com precisão mensal.
    Fonte: https://www.debit.com.br/tabelas/tabela-completa.php?indice=cdi
    
    - De lá extraí também os dados de IPCA(IBGE) (variação mensal) desde o ano de 1981, com precisão mensal.
    Fonte: https://debit.com.br/tabelas/tabela-completa.php?indice=ipca
    

- Yahoo Finace:
    Também utilizaremos como benchmark o indice bovespa (ibovespa) para FI que negociam ações. Para isso recolhemos dados
    do site da Yahoo Finance, com precisão diária.
    Fonte: https://br.financas.yahoo.com/quote/%5EBVSP/history/

### Portal Dados Abertos CVM

O site da CVM disponibiliza diversas planilhas em csv, separadas por ano e mês. Para realizar de forma rápida sua extração, criamos o código de web scraping mostrado a seguir:

In [ ]:
from urllib.request import urlopen, Request

import requests
from bs4 import BeautifulSoup as bs

import zipfile
import os

URL = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'
HIST_URL = 'HIST/'
FILETYPE = '.csv'
HIST_FILETYPE = '.zip'


def get_soup(url):
    return bs(requests.get(url).text, 'html.parser')


for link in get_soup(URL).findAll('a'):
    file_link = link.get('href')
    if FILETYPE in file_link:
        print('Baixando', file_link)
        with open(link.text, 'wb') as file:
            response = requests.get(URL + file_link)
            file.write(response.content)

    if HIST_URL in file_link:
        for hist_link in get_soup(URL + HIST_URL).findAll('a'):
            file_link = hist_link.get('href')
            if HIST_FILETYPE in file_link:
                print('Baixando e descomprimindo', file_link)
                remoteZip = urlopen(Request(URL + HIST_URL + file_link))
                file_name = file_link
                local_file = open(file_name, 'wb')
                local_file.write(remoteZip.read())
                local_file.close()

                with zipfile.ZipFile(file_name, 'r') as zip_ref:
                    zip_ref.extractall()

                os.remove(file_name)

###### Exemplo da database da CVM
 

In [1]:
from tabulate import tabulate
import pandas as pd

cvm_df = pd.read_csv('inf_diario_fi_202106.csv', delimiter =';')
cvm_df = cvm_df.iloc[:10 , :-1]


print(tabulate(cvm_df, headers='keys', tablefmt='psql', showindex=False))

+------------+--------------------+-------------+-------------+------------+-----------------+-------------+------------+
| TP_FUNDO   | CNPJ_FUNDO         | DT_COMPTC   |    VL_TOTAL |   VL_QUOTA |   VL_PATRIM_LIQ |   CAPTC_DIA |   RESG_DIA |
|------------+--------------------+-------------+-------------+------------+-----------------+-------------+------------|
| FI         | 00.017.024/0001-53 | 2021-06-01  | 1.07311e+06 |    27.4711 |     1.0769e+06  |           0 |       0    |
| FI         | 00.017.024/0001-53 | 2021-06-02  | 1.07326e+06 |    27.4725 |     1.07695e+06 |           0 |       0    |
| FI         | 00.017.024/0001-53 | 2021-06-04  | 1.07341e+06 |    27.4742 |     1.07702e+06 |           0 |       0    |
| FI         | 00.017.024/0001-53 | 2021-06-07  | 1.07355e+06 |    27.4756 |     1.07707e+06 |           0 |       0    |
| FI         | 00.017.024/0001-53 | 2021-06-08  | 1.07339e+06 |    27.4771 |     1.07713e+06 |           0 |       0    |
| FI         | 00.017.02

### financial.io

Graças ao api da financial.io pude extrair mais informações de cada fundo, utilizando seu CNPJ (previamente extraido do portal de dados da CVM). Abaixo segue o código para a extração desses dados para uma única tabela, a 'unique_fi_info.csv'

In [ ]:
import requests
import pandas as pd
import os
import json
import re
import datetime


def now():
    now = datetime.datetime.now()
    now_string = f'{now.hour}:{now.minute}:{now.second}'
    return now_string


# Gerando unique_fi.csv, uma tabela com todos os CNPJs únicos da database da CVM:
files = [f for f in os.listdir('.') if os.path.isfile(f) and '.csv' in f]

if 'unique_fi.csv' not in files:
    files.reverse()
    cnpj_df = pd.DataFrame(columns=['CNPJ_FUNDO'])

    for f in files:
        df = pd.read_csv(f, delimiter=';')
        df.drop(df.columns.difference(['CNPJ_FUNDO']), 1, inplace=True)
        df.drop_duplicates(inplace=True)
        cnpj_df = pd.concat([cnpj_df, df])

    cnpj_df.drop_duplicates(inplace=True)
    cnpj_df.to_csv('unique_fi.csv', index=False)

    
# Coletando dados para cada fundo a partir de finantialdata.io:
auth = json.dumps({'usuario': 'manutencao.contas@gmail.com', 'senha': '@eBy5xTZea9fu9n'})
token = requests.post(
    "https://api.financialdata.io/token",
    data=auth,
    headers={'content-type': 'application/json; charset=utf-8',
             'data-type': 'text'}
)


# Criando o 'unique_fi_info.csv' pela primeira vez ou acrescentando dados caso ele já exista:
if 'unique_fi_info.csv' not in files:
    unique_fi_info_df = pd.DataFrame(columns=["CNPJ_FUNDO", 'nome', 'nomeAbreviado', 'dataConstituicao',
                                              'dataCancelamento', 'situacao', 'classe',
                                              'investidorQualificado', 'textoTaxaPerformance',
                                              'nomeAdministrador', 'nomeGestor'])
    unique_fi_info_df.to_csv('unique_fi_info.csv', index=False)
else:
    unique_fi_info_df = pd.read_csv('unique_fi_info.csv')

index = 0
unique_fi_df = pd.read_csv('unique_fi.csv')
unique_fi_df = unique_fi_df.values.tolist()


while index < len(unique_fi_df):
    cnpj = unique_fi_df[index][0]

    if unique_fi_info_df.empty or index not in unique_fi_info_df.index\
            or unique_fi_info_df.at[index, "CNPJ_FUNDO"] != cnpj:
        print(f'{now()} - Colecting data for {cnpj}')

        cnpj_n = re.sub("[^0-9]", "", cnpj)


        try:
            fi_info = requests.get(
                f"https://api.financialdata.io/fundos/{cnpj_n}",
                data=auth,
                headers={'Authorization': 'Bearer ' + token.text}
            )

            if fi_info.status_code == 200:

                fi_df = pd.json_normalize(fi_info.json())

                fi_df.rename(columns={'cnpj': 'CNPJ_FUNDO'}, inplace=True)

                fi_df.drop(fi_df.columns.difference(["CNPJ_FUNDO", 'nome', 'nomeAbreviado',
                                                     'dataConstituicao', 'dataCancelamento',
                                                     'situacao', 'classe', 'investidorQualificado',
                                                     'textoTaxaPerformance', 'nomeAdministrador',
                                                     'nomeGestor']),
                           1,
                           inplace=True)

                unique_fi_info_df = unique_fi_info_df.append(fi_df, ignore_index=True)
                pd.set_option('display.max_columns', None)

                print(f'{now()} - Data successfully collected, FI name is {unique_fi_info_df.at[index, "nome"]}')
            else:
                print(f'{now()} - {cnpj} não encontrado')
                index -=1

        except requests.exceptions.ConnectionError as e:
            print(f'{now()} - finantialdata.io not responding when asked for {cnpj}')
            print(e)
            index -= 1

        # Salvando dados a cada 10 novos fundos recebidos:
        if (index / 10).is_integer():
            unique_fi_info_df.to_csv('unique_fi_info.csv', index=False)

        # Requerindo um novo token a cada 1000 requests (e assim evitando problemas de autenticação):
        if (index / 1000).is_integer():
            token = requests.post(
                "https://api.financialdata.io/token",
                data=auth,
                headers={'content-type': 'application/json; charset=utf-8',
                         'data-type': 'text'}
            )
    index += 1

# Salvando database pela última vez
unique_fi_info_df.to_csv('unique_fi_info.csv', index=False)


###### Exemplo da database de informações dos fundos:

In [2]:
from tabulate import tabulate
import pandas as pd

info_df = pd.read_csv('unique_fi_info.csv')

info_df.drop(info_df.columns.difference(['CNPJ_FUNDO', 'nomeAbreviado', 'classe', 'situacao']),
            1,
            inplace=True
    )

info_df = info_df.iloc[:10]


print(tabulate(info_df, headers='keys', tablefmt='psql', showindex=False))

+--------------------+---------------------------------------------+-------------------------+---------------------+
| CNPJ_FUNDO         | nomeAbreviado                               | situacao                | classe              |
|--------------------+---------------------------------------------+-------------------------+---------------------|
| 00.017.024/0001-53 | FI RF EXPONENCIAL                           | CANCELADA               | nan                 |
| 00.068.305/0001-35 | FIC FI CAIXA EMPREENDER RF LP               | CANCELADA               | nan                 |
| 00.071.477/0001-68 | BB RF CURTO PRAZO AUTOMÁTICO EMPRESA FIC FI | EM FUNCIONAMENTO NORMAL | Fundo de Renda Fixa |
| 00.073.041/0001-08 | BB BESC RF PRÁTICO CP FI                    | CANCELADA               | nan                 |
| 00.083.181/0001-67 | OPP I FIA BDR NÍVEL I IE                    | EM FUNCIONAMENTO NORMAL | Fundo de Ações      |
| 00.089.915/0001-15 | AMARIL FRANKLIN FIC FIM                  

### Debit.com.br

A partir do site da debit extraí via web scraping a tabela com todas as variações do CDI desde 1986:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime

URL = 'https://www.debit.com.br/tabelas/tabela-completa.php?indice=cdi'


def get_soup(url):
    return bs(requests.get(url).text, 'html.parser')


# cdi_df = pd.DataFrame(columns=['date', 'percentage_variation'])


def get_cdi():
    all_tds = get_soup(URL).findAll('td')
    cdi_list = []
    i = 0
    for info in all_tds:
        info_class = info.get('class')
        if info_class == ['mdl-data-table__cell--non-numeric']:
            date = info.text
            variation = all_tds[i+1].text

            cdi_list.append([date, variation.replace(',', '.')])
            pass

        i += 1
    cdi_df = pd.DataFrame(data=cdi_list,columns=['date', 'percentage_variation'])
    cdi_df.to_csv('cdi.csv', index=False)


def covert_dates():
    cdi_df = pd.read_csv('cdi.csv')
    i = 0
    while i < len(cdi_df):
        date = datetime.strptime(cdi_df.at[i, 'date'], '%m/%Y')
        date = datetime.strftime(date, "%Y-%m-%d")
        cdi_df.at[i, 'date'] = date
        i += 1

    cdi_df.to_csv('cdi.csv', index=False)



get_cdi()
covert_dates()


###### Exemplo da database de CDI

In [2]:
from tabulate import tabulate
import pandas as pd

cdi_df = pd.read_csv('cdi.csv', delimiter =',')
cdi_df = cdi_df.tail(10)


print(tabulate(cdi_df, headers='keys', tablefmt='psql', showindex=False))

+------------+------------------------+
| date       |   percentage_variation |
|------------+------------------------|
| 2020-09-01 |                   0.16 |
| 2020-10-01 |                   0.16 |
| 2020-11-01 |                   0.15 |
| 2020-12-01 |                   0.16 |
| 2021-01-01 |                   0.15 |
| 2021-02-01 |                   0.13 |
| 2021-03-01 |                   0.2  |
| 2021-04-01 |                   0.21 |
| 2021-05-01 |                   0.27 |
| 2021-06-01 |                   0.31 |
+------------+------------------------+


###### Do mesmo site retirei as informações relativas ao IPCA:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime

URL = 'https://debit.com.br/tabelas/tabela-completa.php?indice=ipca'


def get_soup(url):
    return bs(requests.get(url).text, 'html.parser')




def get_ipca():
    all_tds = get_soup(URL).findAll('td')
    ipca_list = []
    i = 0
    for info in all_tds:
        info_class = info.get('class')
        if info_class == ['mdl-data-table__cell--non-numeric']:
            date = info.text
            variation = all_tds[i+1].text

            ipca_list.append([date, variation.replace(',', '.')])
            pass

        i += 1
    ipca_df = pd.DataFrame(data=ipca_list,columns=['date', 'ipca_percentage_variation'])
    ipca_df.to_csv('ipca.csv', index=False)


def covert_dates():
    ipca_df = pd.read_csv('ipca.csv')
    i = 0
    while i < len(ipca_df):
        date = datetime.strptime(ipca_df.at[i, 'date'], '%m/%Y')
        date = datetime.strftime(date, "%Y-%m-%d")
        ipca_df.at[i, 'date'] = date
        i += 1

    ipca_df.to_csv('ipca.csv', index=False)



get_ipca()
covert_dates()


###### Exemplo da database do IPCA

In [1]:
from tabulate import tabulate
import pandas as pd

ipca_df = pd.read_csv('ipca.csv', delimiter =',')
ipca_df = ipca_df.tail(10)


print(tabulate(ipca_df, headers='keys', tablefmt='psql', showindex=False))

+------------+-----------------------------+
| date       |   ipca_percentage_variation |
|------------+-----------------------------|
| 2020-09-01 |                        0.64 |
| 2020-10-01 |                        0.86 |
| 2020-11-01 |                        0.89 |
| 2020-12-01 |                        1.35 |
| 2021-01-01 |                        0.25 |
| 2021-02-01 |                        0.86 |
| 2021-03-01 |                        0.93 |
| 2021-04-01 |                        0.31 |
| 2021-05-01 |                        0.83 |
| 2021-06-01 |                        0.53 |
+------------+-----------------------------+


### Yahoo Finace:

No site de finanças do yahoo, conseguimos baixar com apenas um clique os dados diários do IBOV.

###### Exemplo da database do IBOV:

In [3]:
from tabulate import tabulate
import pandas as pd

ibov_df = pd.read_csv('ibov.csv', delimiter =',')
ibov_df = ibov_df.tail(10)


print(tabulate(ibov_df, headers='keys', tablefmt='psql', showindex=False))

+------------+--------+--------+--------+---------+-------------+------------+
| Date       |   Open |   High |    Low |   Close |   Adj Close |     Volume |
|------------+--------+--------+--------+---------+-------------+------------|
| 2021-07-19 | 125958 | 125958 | 123317 |  124395 |      124395 | 9.2102e+06 |
| 2021-07-20 | 124395 | 125631 | 123631 |  125401 |      125401 | 7.6275e+06 |
| 2021-07-21 | 125404 | 126112 | 125247 |  125929 |      125929 | 7.1489e+06 |
| 2021-07-22 | 125930 | 126428 | 125416 |  126147 |      126147 | 6.4804e+06 |
| 2021-07-23 | 126140 | 126204 | 124422 |  125053 |      125053 | 6.0518e+06 |
| 2021-07-26 | 125058 | 126214 | 125006 |  126004 |      126004 | 6.3255e+06 |
| 2021-07-27 | 126004 | 126026 | 123670 |  124612 |      124612 | 7.3138e+06 |
| 2021-07-28 | 124615 | 126712 | 124542 |  126286 |      126286 | 9.0382e+06 |
| 2021-07-29 | 126285 | 126476 | 124917 |  125675 |      125675 | 7.4884e+06 |
| 2021-07-30 | 125672 | 125673 | 121748 |  121801 | 

### ETL concluída, quais os proximos passos?

###### Agora que finalizamos a primeira etapa do projeto, chega a hora de analizar a fundo o cenário dos fundos de investimento brasileiros. Algumas perguntas ficam no ar:

- Qual a duração média dos fundos que não estão mais ativos? Entender mais sobre o ciclo de vida dos fundos brasileiros
  é essencial para melhor definirmos essa classe de ivestimento;

- Quantos dos fundos ativos performam acima do seu benchmark? É realmente lucrativo apostar nos fundos ao invés de apenas
  investir no IBOV ou CDI?

- Quais fundos melhor performaram ao longo dos anos, e por quanto tempo? Em qualquer cenário existem os vencedores e os não
  tão habilidosos. Saber quais os melhores gestores e fundos nos ajudará a alocarmos melhor nosso suado dinheiro!

###### Essas e outras perguntas serão respondidas na proxima sessão: Análise dos FI Brasileiros.